# Unsupervised Learning

In diesem Lernabschnitt befassen wir uns mit einem kleinen Datenset über Einkaufszentrumskunden, die nach ihrem Einkaufs- und Ausgabeverhalten bewertet und eingestuft werden. Im ersten Schritt visualisieren wir den Datensatz. Um verschiedene Gruppen von Einkaufszentrumskunden zu identifizieren, schauen wir uns zwei der grundlegenden Clustering-Algorithmen an: <mark>**K-Means**</mark> und <mark>**DBSCAN**</mark>.

In [ ]:
from IPython.display import Image
from google.colab import drive
drive.mount('/content/drive')
%cd drive/
%cd MyDrive/

### Import von Bibliotheken und Daten

Zuerst importieren wir die Bibliotheken, die wir in diesem Lernabschnitt verwenden werden.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

Mit Hilfe der Bibliothek <code>pandas</code>, die Sie bereits kennen gelernt haben, werden die Daten geladen:

In [ ]:
df = pd.read_csv("mall_customers.csv")
df.head()

## Daten Exploration

Um fehlerhafte oder unvollständige Daten erkennen zu können, nutzen wir die Funktion <code>info()</code>.

In [ ]:
df.info()


Unnötige Spalten werden entfernt und die Spaltenbezeichnungen Jahreseinkommen und Ausgabenbewertung werden gekürzt:

In [ ]:
df = df.drop('CustomerID', axis=1)
df.rename(index=str, columns={'Annual Income (k$)': 'Income', 'Spending Score (1-100)': 'Score'}, inplace=True)
df.head()

Jede Spalte repräsentiert einen relevanten Aspekt des Kunden. Die Beziehung zwischen den einzelnen Kundenaspekten können wir über die Bibliothek <code>seaborn</code> visualisieren. Hier eignet sich die Funktion <code>pairplot</code>, die ein Diagramm für jedes numerische Attributpaar erstellt:

In [ ]:
sns.pairplot(df, hue='Gender', aspect=2)
plt.show()

Das Argument "hue" wendet die Farbe auf die Datenpunkte an, die der Spalte "Gender" entsprechen. Mit Hilfe des Arguments "aspect" wird das Seitenverhältnis der Diagramme festgelegt. 

Wir entfernen die Spalte "Gender" aus dem Datenset, da es sich nicht um numerische Werte handelt und wir sie für die Clustering-Algorithmen nicht mehr verwenden. Nun weisen wir den Dataframe der Variablen X zu:

In [ ]:
X = df.drop(['Gender'], axis=1)

## Clustering Alogrithmen

Nun kommen wir zum spannenden Teil des Lernabschnitts. Wir nutzen verschiedene Algorithmen um ähnliche Datenpunkte in Gruppen zusammenzufassen. 

### K-Means

Mit Hilfe der Zielzahl "k" wird die Anzahl der Zentroide festgelegt, die man im Datensatz finden möchte. Ein Zentroid repräsentiert das Zentrum eines Clusters. Zuerst werden k Datenpunkte zufällig aus allen verfügbaren Datenpunkten gezogen und als vorläufige Zentroide zugewiesen. Im nächsten Schritt ordnet der Algorithmus jeden der verbleibenden Datenpunkte einem der Cluster zu, ausgehend vom nächstgelegenen Zentroid. Anschließend werden die Zentroide neu berechnet, so dass sie die tatsächlichen Zentren der aktuellen Cluster darstellen. Die Zuweisung von Datenpunkten zu Clustern und der anschließenden Neuberechnung der Zentroide wird so lange wiederholt, bis die Lage der Zentroide final bestimmt ist. An diesem Punkt gelten die Zentroide als konvergiert und der Algorithmus wird beendet.

Mit der Bibliothek <code>sklearn</code> lässt sich die Funktion KMeans wie folgt anwenden:

In [ ]:
km3 = KMeans(n_clusters=3).fit(X)
print(km3.labels_)

Die obige Liste gibt an, welchem der 3 Cluster jeder Datenpunkt zugeordnet wurde. Um unsere Ergebnisse zu visualisieren, müssen wir zunächst die Clusterzuordnung zu unseren Daten <code>X</code> als zusätzliche Spalte hinzufügen. Um die Datenpunkte entsprechend ihrer Clusterzuordnung einzufärben, verwenden wir wieder das <code>hue</code> Argument, wie wir es zuvor zur Unterscheidung der Geschlechter getan haben. In einem zweidimensionalen Diagramm, also müssen wir **Age**, **Income** und **Score** wählen. Hier verwenden wir nur **Income** und **Score**:

In [ ]:
X['Labels'] = km3.labels_
sns.scatterplot(x=X['Income'], y=X['Score'], hue=X['Labels'], palette=sns.color_palette('hls', 3))
plt.title('KMeans with 3 Clusters')
plt.show()

Unser Algorithmus funktioniert und die Clusterzuordnungen scheinen plausibel zu sein. Wenn man sich jedoch die Verteilung der Punkte ansieht, sollte es 5 statt 3 Cluster geben.

<mark>**Aufgabe:**</mark> Wenden Sie nun K-means mit 5 Clustern an:

In [ ]:
# Füge Sie hier Ihren Code ein.

Eine andere Möglichkeit, die Daten zu visualisieren, ist die Verwendung mit Hilfe von <code>swarmplot</code>:

In [ ]:
plt.figure(figsize=(8,4))
sns.swarmplot(x=X['Labels'], y=X['Income'])
plt.title("Labels According to Annual Income")
plt.show()

Aufgabe: Bauen Sie einen Swarmplot Score vs. Labels auf dem gleichen Datenset.

In [ ]:
# Füge Sie hier Ihren Code ein.

Da sich die Bestimmung der richtigen Anzahl an Clustern mit dem bloßen Auge bei steigender Spaltenanzahl als schwierig erweist, gibt es eine systematischere Methode zur Bestimmung von <code>k</code>.

Die <code>sklearn</code> -Implementierung des K-Means-Algorithmus liefert zusätzlich zu den Cluster-Labels auch die Summe der Distanzen von Datenpunkten zu ihren jeweiligen Zentroiden als <code>inertia_</code> (Trägheit). Damit kann eine Reihe verschiedener Werte für <code>k</code> ausprobiert und die Qualität jedes Ergebnisses mit diesem Trägheitsparameter bewertet werden:

In [ ]:
clusters = []

for i in range(1, 11):
    km = KMeans(n_clusters=i).fit(X)
    clusters.append(km.inertia_)
    
sns.lineplot(x=list(range(1, 11)), y=clusters)
plt.title('Searching for Elbow')
plt.xlabel('Clusters')
plt.ylabel('Inertia')
plt.show()

Die Trägheit nimmt mit zunehmender Anzahl von Clustern kontinuierlich ab. Im Extremfall stellt jeder einzelne Datenpunkt einen Schwerpunkt dar und die Trägheit verschwindet, da jeder Punkt den Abstand 0 zu sich selbst hat. Wenn jedoch die Anzahl an Clustern der Anzahl an Datenpunkten entspricht, verfehlt das Clustering seinen Zweck.
Um eine geeignete Anzahl an Clustern zu wählen, hilft der sogenannte <Mark>**Ellenbogenpunkt**</mark>. Dieser lässt sich im Diagramm visuell ablesen und die Anzahl der Cluster zu bestimmen.

### DBSCAN

Eine weitere Clustering-Methode stellt DBSCAN dar. DBSCAN steht für "Density-Based Spatial Clustering of Applications with Noise". Diese Technik ist einer der gängigsten Clustering-Algorithmen, die auf der Dichte von Objekten basieren. Es gibt bestimmte Geometrien der Datenpunktverteilung, die DBSCAN erfolgreich verarbeiten kann, wo K-Means versagt:

In [ ]:
Image("DBSCAN_vs_KMeans.png")

DBSCAN geht davon aus, dass ein Punkt, der zu einem Cluster gehört, in der Nähe vieler anderer Punkte desselben Clusters liegen sollte. Daher identifiziert der Algorithmus zunächst sogenannte <mark>**Kernpunkte</mark>, die in der Nähe vieler anderer Punkte liegen, d. h. mindestens <code>min_samples</code>, also die Mindestanzahl an Nachbarn, die einen bestimmten Radius <code>eps</code> haben. Diese beiden Parameter repräsentieren die beiden wichtigsten Parameter, die für diesen Algorithmus eingestellt werden müssen.

Im zweiten Schritt wird ein Kernpunkt zufällig ausgewählt und einem Cluster zugewiesen. Alle anderen Punkte in seiner Nähe werden dann demselben Cluster hinzugefügt. Dann werden alle Punkte in der Nähe *dieser* Punkte hinzugefügt, und so weiter, bis keine weiteren Punkte in der Nähe gefunden werden können, die nicht bereits zum Cluster gehören. Damit ist unser erster Cluster abgeschlossen.

Der obige Vorgang wird dann für einen weiteren Kernpunkt wiederholt, der noch übrig ist, bis jeder Punkt, der einem Cluster zugeordnet werden kann, zugeordnet wurde. Jeder verbleibende Punkt wird als <mark>**Ausreißer</mark> betrachtet und erhält das Cluster-Label <code>-1</code>. 

Hier ist ein Beispielcode für <code>eps=6</code> und <code>min_samples=6</code>:

In [ ]:
db = DBSCAN(eps=6, min_samples=6).fit(X)

X['Labels'] = db.labels_
sns.scatterplot(x=X['Income'], y=X['Score'], hue=X['Labels'], 
                palette=sns.color_palette('tab10', np.unique(db.labels_).shape[0]))
plt.title('DBSCAN mit epsilon 6, min_samples 6')
plt.show()

Wie wir sehen können, sind die Parameter noch nicht sehr gut abgestimmt, da die meisten Punkte noch keinen Clustern zugeordnet wurden.

<mark>**Aufgabe:**</mark> Probieren Sie verschiedene Parameter für DBSCAN aus, bis Sie ein zufriedenstellendes Ergebnis erhalten. Hinweis: Selbst mit optimierten Parametern schneidet DBSCAN bei diesem speziellen Datensatz nicht so gut ab wie K-Means, da die Cluster recht unterschiedliche Dichten aufweisen.

In [ ]:
# Füge Sie hier Ihren Code ein.